In [ ]:
import gc
import sys
import cv2
import progressbar as pb
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from common import *

In [ ]:
# Set File Name
fileName = "cs10.csv"

In [ ]:
# Load CSV & Calculate necessary frames
print("[1/4] Loading CSV ( " + fileName + " ) ...")
nx = getNx(fileName)
print("[2/4] Calculating necessary frames...")
frameIdx = getNecessaryFrameIdx(nx)
frameLen = len(frameIdx)

In [ ]:
# Calculate graphs
print("[3/4] Creating graphs and converting to cv format...")
frames = []
barIdx = 0
tr = int((nx.shape[1] - 1) / 60)
with pb.ProgressBar(max_value=frameLen) as bar:
    for p in frameIdx:
        pltLegend = []
        pltFigure = plt.figure()
        pltImage = BytesIO()
        for i in range(0, tr):
            pltLegend.append(str(i + 1) + ' TRP')  # Tx-Rx Pair
            pltPlot = plt.plot(nx[p, (tr * 30 + 1 + 30 * i):(tr * 30 + 31 + 30 * i)], figure=pltFigure)
            plt.xlabel('Subcarriers Group', figure=pltFigure)
            plt.ylabel('Phase [db]', figure=pltFigure)
        pltFigure.legend(pltLegend)
        plt.ylim(0, 20)
        pltFigure.savefig(pltImage, format='png')
        plt.close(pltFigure)
        pltImage.seek(0)
        cvImage = cv2.imdecode(np.asarray(
            bytearray(pltImage.read()), dtype=np.uint8), cv2.IMREAD_COLOR)
        frames.append(cvImage)
        barIdx += 1
        bar.update(barIdx)

In [ ]:
# Save as mp4 video
videoName = conf.VID_NAME.format(
    fileName[:fileName.index('.')], 'phase')
print("[4/4] Saving in " + videoName + " ...")
saveVideo(videoName, frames)

In [ ]:
#Finished